In [1]:
! pip install .\dist\quick_sentiments-0.1.6-py3-none-any.whl


Processing c:\users\meala\documents\quick_sentiments\dist\quick_sentiments-0.1.6-py3-none-any.whl
  Using cached numpy-2.3.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.3.1-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached gensim-4.3.0-cp311-cp311-win_amd64.whl.metadata (8.6 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached xgboost-3.0.2-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached scipy-1.16.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached smart_open-7.3.0.post1-py3-none-any.whl.metadata (24 kB)
  Using cached FuzzyTM-2.0.9-py3-none-any.whl.metadata (7.9 kB)
  Using cached Cython-0.29.32-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pyFUME-

In [1]:
from quick_sentiments import pre_process
from quick_sentiments import run_pipeline
from quick_sentiments import make_predictions

NLTK data 'punkt' already present.
NLTK data 'stopwords' already present.
NLTK data 'wordnet' downloaded.
NLTK data 'omw-1.4' downloaded.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\meala\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\meala\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
import polars as pl

In [5]:
df = pl.read_csv("quick_sentiments/demo/training_data/train.csv")

In [6]:
df.head(5)

movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
str,str,bool,str,str
"""marvelous_pira…","""Benjamin Henry…",false,"""Henry Selick’s…","""POSITIVE"""
"""tony_montana_f…","""Felicia Lopez""",false,"""With a cast th…","""NEGATIVE"""
"""darth_vader_ka…","""Mr. Charles Bu…",true,"""Creed II does …","""POSITIVE"""
"""lara_croft_gli…","""Ryan Barrett""",false,"""I know what yo…","""POSITIVE"""
"""jason_bourne_s…","""Alexander Glov…",false,"""Director Ferna…","""POSITIVE"""


In [7]:
pre_process(df["reviewText"][0])

'henry selicks first movie since 2009s coraline fifth stopmotion masterpiece'

In [8]:
df = df.sample(fraction=0.1, with_replacement=False)
df = df.with_columns(
    pl.col("reviewText").map_elements(lambda x: pre_process(x, remove_brackets=True)).alias("processed")  #add inside the map_elements
)
df.head(10)

movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,processed
str,str,bool,str,str,str
"""mythical_john_…","""Patrick Poole""",false,"""Is it ironic o…","""NEGATIVE""","""ironic par cou…"
"""tyler_durden_h…","""Jennifer Clayt…",true,"""There is a lot…","""NEGATIVE""","""lot skill disp…"
"""secret_marty_m…","""Patrick Arias""",true,"""A satisfying m…","""POSITIVE""","""satisfying mix…"
"""celestial_king…","""Larry Patterso…",false,"""It deals with …","""POSITIVE""","""deal unpleasan…"
"""intrigue_vivid…","""Molly Cabrera""",true,"""Writer-directo…","""NEGATIVE""","""writerdirector…"
"""james_bond_iro…","""James Newman""",false,"""While the pict…","""NEGATIVE""","""picture superi…"
"""marty_mcfly_si…","""Maria Park""",true,"""High-Rise is a…","""NEGATIVE""","""highrise deepl…"
"""harmony_rick_b…","""Alejandra Lope…",false,"""Barrymore's mo…","""POSITIVE""","""barrymore movi…"
"""quest_enchant_…","""Jeffery Baird""",false,"""A movie that m…","""NEGATIVE""","""movie manufact…"


In [9]:
dt= run_pipeline(
    vectorizer_name="BOW", # BOW, tf, tfidf, wv
    model_name="logit", # logit, rf, XGB .#XGB takes long time, can not recommend using it on normal case
    df=df,
    text_column_name="reviewText",  # this is the column name of the text data, 
    sentiment_column_name = "sentiment",
    perform_tuning = True # make this true if you want to perform hyperparameter tuning, it will take longer time and 
                            # may run out of memory if the dataset is large,
)


--- Running Pipeline for Bow + Logit ---
Labels encoded: Original -> ['NEGATIVE' 'POSITIVE'], Encoded -> [0 1]
1. Vectorizing entire dataset (X)...
   - Generating Bag-of-Words features...
2. Splitting data into train/test...
3. Training and predicting...
   - Starting Logistic Regression training with GridSearchCV for hyperparameter tuning...
   - Using default parameter grid for tuning: {'solver': ['liblinear', 'lbfgs'], 'C': [0.1, 1.0, 10.0], 'class_weight': [None, 'balanced'], 'max_iter': [500, 1000]}
Fitting 5 folds for each of 24 candidates, totalling 120 fits

   - Best Hyperparameters found:
{'C': 1.0, 'class_weight': 'balanced', 'max_iter': 500, 'solver': 'lbfgs'}
   - Best Cross-Validation Score (F1-weighted): 0.7491
Best model parameters: {'C': 1.0, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 500, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'to

In [10]:
new_data = pl.read_csv("quick_sentiments/demo/new_data/test.csv",encoding='ISO-8859-1') #keep your file here
print(new_data.shape)
new_data= new_data.sample(fraction=0.25, shuffle=True, seed=42)
print(new_data.shape)

(55315, 4)
(13828, 4)


In [11]:
new_data = new_data.with_columns(
    pl.col("reviewText").map_elements(lambda x: pre_process(x, remove_brackets=True)).alias("processed")  #add inside the map_elements
)
new_data.head(10)

movieid,reviewerName,isTopCritic,reviewText,processed
str,str,bool,str,str
"""frodo_baggins_…","""Toni Vaughn""",false,"""The result is …","""result unsettl…"
"""stardust_john_…","""Carol Jennings…",false,"""Think twice ab…","""think twice ge…"
"""hermione_grang…","""Tara Huang""",true,"""A film that's …","""film bloody wo…"
"""astonish_valia…","""Shelley Murill…",false,"""...a decent se…","""decent setup e…"
"""indiana_jones_…","""Daniel Bond""",false,"""Inspiring? Not…","""inspiring lame…"
"""glimmer_hannib…","""Mrs. Nicole Fl…",false,"""This perfectly…","""perfectly exec…"
"""brave_anakin_s…","""Melissa Harrin…",false,"""The Richard Cu…","""richard curtis…"
"""katniss_everde…","""Mckenzie Ortiz…",false,"""A fun-filled a…","""funfilled afte…"
"""secret_magic_j…","""Samantha Ware""",false,"""I'm still not …","""still sure sup…"


In [12]:
make_predictions(
    new_data=new_data,
    text_column_name="processed",
    vectorizer=dt["vectorizer_object"],
    best_model=dt["model_object"],
    label_encoder=dt["label_encoder"],
    prediction_column_name="sentiment_predictions"  # Optional custom name
)

movieid,reviewerName,isTopCritic,reviewText,processed,sentiment_predictions
str,str,bool,str,str,str
"""frodo_baggins_…","""Toni Vaughn""",false,"""The result is …","""result unsettl…","""POSITIVE"""
"""stardust_john_…","""Carol Jennings…",false,"""Think twice ab…","""think twice ge…","""POSITIVE"""
"""hermione_grang…","""Tara Huang""",true,"""A film that's …","""film bloody wo…","""POSITIVE"""
"""astonish_valia…","""Shelley Murill…",false,"""...a decent se…","""decent setup e…","""NEGATIVE"""
"""indiana_jones_…","""Daniel Bond""",false,"""Inspiring? Not…","""inspiring lame…","""NEGATIVE"""
"""glimmer_hannib…","""Mrs. Nicole Fl…",false,"""This perfectly…","""perfectly exec…","""POSITIVE"""
"""brave_anakin_s…","""Melissa Harrin…",false,"""The Richard Cu…","""richard curtis…","""POSITIVE"""
"""katniss_everde…","""Mckenzie Ortiz…",false,"""A fun-filled a…","""funfilled afte…","""NEGATIVE"""
"""secret_magic_j…","""Samantha Ware""",false,"""I'm still not …","""still sure sup…","""NEGATIVE"""
